In [48]:
import numpy as np
from qiskit import QuantumCircuit, transpile, assemble, Aer, IBMQ
from qiskit.quantum_info.operators import Operator

# C1 and C2

In [49]:
#parameters: w1,w2=0.5   e1,e2=+/-1     cs=1   u=0.2   
# x=w1(1+e1*u/cs^2) = 0.6         equation 3 link 1
# y=w2(1+e2*u/cs^2) = 0.4         equation 3 link 2
x = 0.6
y = 0.4


# lambda1= x +/- i Sqrt(1-x^2)      
# lambda2= y +/- i Sqrt(1-y^2)         
# positive for C1, negative for C2

lambda1 = np.round(np.log(complex(x, np.sqrt(1-x**2))), 10)  
lambda2 = np.round(np.log(complex(y, np.sqrt(1-y**2))), 10)

print(lambda1,lambda2) #log of complex number returns imaginary number

0.927295218j 1.1592794807j


In [50]:
def createC(lambda1, lambda2, isC1 = True):
    
    if not isC1:
        lambda1=np.conj(lambda1)
        lambda2=np.conj(lambda2)
    
    lambda1=lambda1.imag#temporary
    lambda2=lambda2.imag#temporary
    
    qc=QuantumCircuit(2)
    
    #phase1 -> x -> phase1 -> x -----ctrl=0
    qc.cp(lambda1, 0 ,1 , ctrl_state = 0)
    qc.cx(0, 1, ctrl_state = 0)
    qc.cp(lambda1, 0, 1, ctrl_state = 0)
    qc.cx(0, 1, ctrl_state = 0)
    
    #phase2 -> x -> phase2 -> x -----ctrl=1 is default
    qc.cp(lambda2, 0, 1)
    qc.cx(0, 1)
    qc.cp(lambda2, 0, 1)
    qc.cx(0, 1)
    
    return qc

In [51]:
C1 = createC(lambda1, lambda2, True)
C1.draw()

q_0: ─o────────────o───o────────────o───■────────────■───■────────────■──
      │P(0.9273) ┌─┴─┐ │P(0.9273) ┌─┴─┐ │P(1.1593) ┌─┴─┐ │P(1.1593) ┌─┴─┐
q_1: ─■──────────┤ X ├─■──────────┤ X ├─■──────────┤ X ├─■──────────┤ X ├
                 └───┘            └───┘            └───┘            └───┘

In [52]:
C2 = createC(lambda1, lambda2, False)
C2.draw()

q_0: ─o─────────────o───o─────────────o───■─────────────■───■─────────────■──
      │P(-0.9273) ┌─┴─┐ │P(-0.9273) ┌─┴─┐ │P(-1.1593) ┌─┴─┐ │P(-1.1593) ┌─┴─┐
q_1: ─■───────────┤ X ├─■───────────┤ X ├─■───────────┤ X ├─■───────────┤ X ├
                  └───┘             └───┘             └───┘             └───┘

# L and R gates

In [53]:
n = 4 # 2^number of qubits

#right shift
rshift = np.eye(n,k = -1)
rshift[0][-1] = 1

#left shift
lshift = np.eye(n,k = 1)
lshift[-1][0] = 1

print(rshift,"\n\n", lshift)

[[0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]] 

 [[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]]


In [54]:
R = Operator(rshift)
L = Operator(lshift)